In [73]:
pip install imutils

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [74]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils

In [85]:
def verify_content(image, threshold=0.4):
    height, width, _ = image.shape
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_gray = cv2.GaussianBlur(image_gray, (7, 7), 0)
    # cv2_imshow(image_gray)
    # plt.hist(image_gray.ravel(), 256,[0, 256]); plt.show()
    (T, image_threshold) = cv2.threshold(image_gray, 55, 255, cv2.THRESH_BINARY_INV)
    # cv2_imshow(image_threshold)
    contours = cv2.findContours(image_threshold .copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    image_clone = image.copy()
    cv2.drawContours(image_clone, contours, -1, (255, 0, 0), 2)
    # cv2_imshow(image_clone)
    areas = [cv2.contourArea(contour) for contour in contours]
    (contours, areas) = zip(*sorted(zip(contours, areas), key=lambda a:a[1]))
    image_clone = image.copy()
    cv2.drawContours(image_clone, [contours[-1]], -1, (255, 0, 0), 2)
    # cv2_imshow(image_clone)
    image_clone = image.copy()
    (x, y, w, h) = cv2.boundingRect(contours[-1])
    aspectRatio = h / float(height)
    # print(aspectRatio)
    if aspectRatio > threshold:
        cv2.rectangle(image_clone, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image_clone, "High", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
        # cv2_imshow(image_clone)
        return True
    else:
        cv2.rectangle(image_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(image_clone, "Low", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
        # cv2_imshow(image_clone)
        return False
    # cv2_imshow(image_clone)
   






In [76]:
def smashed_detector(image, threshold = 0.4):
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_gray = cv2.GaussianBlur(image_gray, (7, 7), 0)
    border = cv2.Canny(image_gray, 30, 150)
    contours = cv2.findContours(border.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    for contour in contours:
        perimetro = cv2.arcLength(contour, True)
        aproximacao = cv2.approxPolyDP(contour, 0.02 * perimetro, True)
        if len(aproximacao) == 4:
           cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
           cv2.putText(image, 'Garrafa Quebrada', (10, 30), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
        else:
           cv2.drawContours(image, [contour], -1, (0, 0, 255), 2)
           cv2.putText(image, 'Garrafa Inteira', (10, 30), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
    #cv2.imshow('smashed bottle', image)
    #cv2.waitKey(0)


In [77]:

def white_detector_rgb(image, threshold=0.27):
    lower_white = np.array([200, 200, 200], dtype=np.uint8)
    upper_white = np.array([255, 255, 255], dtype=np.uint8)

    mask_white = cv2.inRange(image, lower_white, upper_white)

    total_pixels = image.shape[0] * image.shape[1]
    white_pixels = np.sum(mask_white) // 255
    white_relation = white_pixels / total_pixels

    return white_relation>= threshold

In [78]:
def out_of_place(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 100)
    lines = cv2.HoughLines(edges, 1, np.pi/180, 51)
    horizontal_lines = 0
    vertical_lines = 0
    ignored_lines = 0
     

    if lines is not None:
        for line in lines:
            _, theta = line[0]
            theta_degrees = np.degrees(theta)

            if 10 <= theta_degrees <= 170:
                horizontal_lines += 1
            elif 80 <= theta_degrees <= 100:
                ignored_lines += 1
            else:
                vertical_lines += 1

        if horizontal_lines >= vertical_lines or ignored_lines == len(lines) or len(lines) == 0:
           on_place = True
        else:
            on_place = False
    else:
        on_place = True  

    return on_place

   
          
     

In [79]:
def black_detector(image, threshold = 0.135):
    bottle_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    bottle_gray = cv2.GaussianBlur(bottle_gray, (7, 7), 0)
    (T, bottle_threshold) = cv2.threshold(bottle_gray, 55, 255, cv2.THRESH_BINARY_INV)
    black_segment = cv2.bitwise_and(image, image, mask=bottle_threshold)
    # cv2_imshow(black_segment)

    total_pixels = image.shape[0] * image.shape[1]
    black_pixels = np.sum(bottle_threshold) // 255
    black_percentage = black_pixels / total_pixels
    # print(f'Percentual de preto: {black_percentage}')

    return black_percentage >= threshold
     
     

In [80]:

def red_detector_rgb(image, threshold=0.4):
    lower_red = np.array([0, 0, 100])
    upper_red = np.array([100, 100, 255]) 
    mask_red_rgb = cv2.inRange(image, lower_red, upper_red)
    contours, _ = cv2.findContours(mask_red_rgb, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    total_area = 0
    for contour in contours:
        total_area += cv2.contourArea(contour)
    total_pixels = image.shape[0] * image.shape[1]
    red_img = total_area / total_pixels

    return red_img >= threshold



In [87]:
def bottom_treatment(img):
    height, width, _ = img.shape
    response = np.zeros(8)

    # Dividindo a imagem em 3 partes
    third_length = width // 3

    
    bottle2 = img[:, third_length:2*third_length]
    

    middle_cap = bottle2[:height//5, :]
    middle_cover = bottle2[170:305, :]
    middle_neck = bottle2[40:185, :]

    #verificando as caracteristicas
    full = black_detector(middle_neck)
    capped = red_detector_rgb(middle_cap, threshold=0.3)
    redCover = red_detector_rgb(middle_cover)
    whiteCover = white_detector_rgb(middle_cover)
    covered = redCover or whiteCover
    unlabeled = black_detector(middle_cover, threshold=0.42)

    if full or capped or covered:
        if not capped:
            response[2] = 1
        if whiteCover:
            response[4] = 1
        if unlabeled:
            response[6] = 1
        if not unlabeled and not out_of_place(middle_cover):
            response[5] = 1
        if full:
            if verify_content(middle_neck):
                response[0] = 1
        else:
            response[1] = 1
    else:
        response[7] = 1

    return response
     



In [88]:
#lendo as imagens7
for i in range (1,78):
  image = cv2.imread(f'train/train_{i}.jpg')
  print(f'train_{i}.jpg')
  #cv2.imshow(f"train/train_{i}.jpg", image) 
  #verify_content(image)
  print(bottom_treatment(image))
  #cv2.waitKey(0)
 

train_1.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]
train_2.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]
train_3.jpg
[1. 0. 1. 0. 0. 1. 0. 0.]
train_4.jpg
[1. 0. 1. 0. 0. 0. 0. 0.]
train_5.jpg
[1. 0. 1. 0. 0. 0. 0. 0.]
train_6.jpg
[1. 0. 1. 0. 0. 0. 0. 0.]
train_7.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]
train_8.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]
train_9.jpg
[0. 0. 0. 0. 0. 0. 0. 1.]
train_10.jpg
[1. 0. 0. 0. 0. 0. 0. 0.]
train_11.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]
train_12.jpg
[1. 0. 1. 0. 0. 0. 0. 0.]
train_13.jpg
[0. 0. 0. 0. 0. 0. 0. 1.]
train_14.jpg
[0. 0. 0. 0. 0. 0. 1. 0.]
train_15.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]
train_16.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]
train_17.jpg
[0. 1. 1. 0. 0. 0. 0. 0.]
train_18.jpg
[1. 0. 0. 0. 0. 1. 0. 0.]
train_19.jpg
[1. 0. 1. 0. 0. 0. 0. 0.]
train_20.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]
train_21.jpg
[1. 0. 0. 0. 0. 0. 0. 0.]
train_22.jpg
[0. 1. 0. 0. 0. 1. 0. 0.]
train_23.jpg
[1. 0. 1. 0. 0. 0. 1. 0.]
train_24.jpg
[1. 0. 0. 0. 0. 0. 0. 0.]
train_25.jpg
[0. 0. 0. 0. 0. 0. 0. 0.]
train_26.jpg
[0. 0. 0. 0. 0. 0. 0.